In [20]:
import urllib.request
import pandas as pd
import json

df = pd.DataFrame(columns=["language", "review", "voted_up", "votes_up", "votes_funny", "weighted_vote_score", "comment_count", "steam_purchase", "received_for_free", "written_during_early_access"])

# Load appids

In [27]:
games = [837910,2514620,2017950,933756,422100,815740,1295120,1336460,781880,914290,2349580,518908,989560,2062020,300642,1951660,577280,818070,2092500,1515920,1284480,1339380,226453,819980,1319750,2128870,1259230,400140,593420,693623,733670,2180180,511920,1019770,756200,206521,334490,987020,1090090,5688,522870,551160,1906710,2306370,1160243,302340,2319700,1393570,1781760,562040,2710300,585,292230,2024440,853887,1853370,2081500,772710,642805,336040,1681290,369530,1684160,1135591,518848,1828230,2063850,1396940,2015840,1588570,1179260,2623440,313560,2424630,1441714,1621650,346470,1105400,3430,566910,2409940,741590,1099240,1857940,2470710,1521070,2314380,2492080,1240300,2545170,1630080,488700,1689060,2479440,1680973,356744,1917610,396620,1736801,1773050,541890,1571260,415960,269890,11500,439700,2057781,2363960,1114990,1351420,966830,1885090,1989230,1932470,1572960,716940,2534660,2460300,2334760,936850,605660,2068190,2294610,839760,2568860,2673080,845870,2663830,1731930,1195620,1239450,1050560,2001900,1592760,364060,204483,590840,397990,1448010,2589650,518949,387530,2285890,295200,779300,1700470,2340170,211,1159240,268101,714080,407150,409257,1803580,1202034,1290911,2106930,2622870,1498130,729230,940670,1245200,2366470,726510,81107,2269020,2575520,1900860,2455730,2587190,2091850,2404470,1694070,2407764,1766690,1378300,367970,1833960,694060,235580,2165600,1630740,409750,535043,1336482,1930310,2279510,2161920,1105160,67250,1115680,774131,1298220,1925030,2611280,785560,409960,312260,2380190,1183670,1587570,2146500,2554360,1442280,949060,230760,1630601,1205470,1714370,2626870,1026970,2530560,524210,42510,2678060,600630,767,26700,1794760,2318880,917240,1750170,2461430,1005180,349020,623140,2276850,1542910,2580740,1198841,387740,1403740,2518800,2150060,1775000,1852251,379544,864790,2377030,1694170,2314470,1022448,2096830,1227453,527402,32110,1101960,1469620,591740,2015500,1339110,1686390,735540,625730,1404042,1269610,1835810,34344,2231840,358250,353570,2344930,249310,1684780,50414,1086160,269470,1553130,1439340,1285590,1902600,2538200,524042,1672490,2255060,584860,1805059,1492090,331160,2350830,2083130,973270,2209560,1520800,960890,2201560,593900,2190320,2321920,346450,2184560,651140,2127090,1280420,2061290,2169690,1102060,2586420,1934790,1787860,1748472,2163450,1830010,271411,2309890,648110,544230,1827010,1936510,2701470,1485291,2080060,1461854,1135310,2386992,1540490,895370,505015,253831,886140,660940,1791710,491550,766100,2275650,627855,1782480,1063080,389670,851318,1841800,1552561,796860,1427591,457210,1695890,874890,357190,1594170,1619880,550400,721146,2181140,1533250,2082290,1288170,2419627,2483020,597383,1043420,538080,959,583570,1117730,410810,526760,759418,668490,441250,1685970,2383600,988970,2194040,442240,492720,2372580,1343250,311190,527810,1437680,1334790,1658130,1363650,454800,1364410,2666920,15190,2021880,905340,1526820,1674420,1649410,1665400,436070,1652200,24038,2169140,952798,1555442,594220,1004919,1004390,343300,2537490,1392690,80728,1453380,1488890,2622500,573560,1364090,81234,1705249,701420,820500,2253050,2014510,783391,2560510,1795180,81634,952970,48804,2381740,1274850,1150790,2551580,445931,1006890,2431400,633830,951820,1746550,718720,1318820,1736802,1965290,1295440,1377580,227020,81472,2114630,1961400,2067790,965680,1430050,1516930,2141700,2482540,1843500,1671310,1117390,2193600,6550,1466650,527403,222603,2003600,35050,1367770,541930,1175560,650350,1601680,1484380,2011390,2003620,2469360,1495370,1615620,1299660,359480,1449580,751670,726550,362670,1341890,1295920,2191660,889921,2385670,206650,442318,1784780,1257950,662840,747740,1132220,1380170,1774345,2262250,1203482,577080,1476040,2108460,2212875,390371,2259010,2089812,2663080,430890,348470,2501590,2108020,937710,439610,1929650,2069930,3010,1110030]

#with open("games.json") as data:
#    d = json.load(data)
#    data.close()

#for i in range(0, d["response"]["game_count"]):
#    games.append(d["response"]["games"][i]["appid"])

# Get reviews

In [28]:
with urllib.request.urlopen("http://store.steampowered.com/appreviews/4000?json=1") as url:
    d = json.load(url)
    url.close()

for i in games:
    # Seed first cursor
    with urllib.request.urlopen("http://store.steampowered.com/appreviews/"+str(i)+"?json=1") as url:
        d = json.load(url)
        url.close()
        
        cursor = d["cursor"]
        for j in d["reviews"]:
            df.loc[len(df)] = [j["language"], j["review"], j["voted_up"], j["votes_up"], j["votes_funny"], j["weighted_vote_score"], j["comment_count"], j["steam_purchase"], j["received_for_free"], j["written_during_early_access"]]
    
    for j in range(1, 100):
        with urllib.request.urlopen("http://store.steampowered.com/appreviews/"+str(i)+"?json=1&cursor="+cursor) as url:
            d = json.load(url)
            url.close()
            if "cursor" in d.keys():
                cursor_new = d["cursor"]
            else:
                break
            
        if cursor_new == cursor:
            break
        else:
            cursor = cursor_new
            for k in d["reviews"]:
                df.loc[len(df)] = [k["language"], k["review"], k["voted_up"], k["votes_up"], k["votes_funny"], k["weighted_vote_score"], k["comment_count"], k["steam_purchase"], k["received_for_free"], k["written_during_early_access"]]
    print(df.shape)

(106, 10)
(106, 10)
(110, 10)
(110, 10)
(128, 10)
(128, 10)
(128, 10)
(130, 10)
(139, 10)
(159, 10)
(159, 10)
(159, 10)
(169, 10)
(169, 10)
(169, 10)
(182, 10)
(193, 10)
(193, 10)
(193, 10)
(193, 10)
(197, 10)
(198, 10)
(198, 10)
(201, 10)
(211, 10)
(211, 10)
(211, 10)
(211, 10)
(211, 10)
(211, 10)
(214, 10)
(215, 10)
(216, 10)
(216, 10)
(216, 10)
(236, 10)
(240, 10)
(242, 10)
(242, 10)
(242, 10)
(242, 10)
(244, 10)
(244, 10)
(255, 10)
(256, 10)
(276, 10)
(276, 10)
(278, 10)
(278, 10)
(278, 10)
(278, 10)
(278, 10)
(298, 10)
(298, 10)
(298, 10)
(298, 10)
(298, 10)
(298, 10)
(301, 10)
(302, 10)
(302, 10)
(303, 10)
(303, 10)
(303, 10)
(303, 10)
(303, 10)
(304, 10)
(304, 10)
(305, 10)
(305, 10)
(305, 10)
(306, 10)
(326, 10)
(327, 10)
(327, 10)
(329, 10)
(349, 10)
(349, 10)
(351, 10)
(352, 10)
(352, 10)
(352, 10)
(352, 10)
(352, 10)
(352, 10)
(354, 10)
(354, 10)
(354, 10)
(354, 10)
(355, 10)
(359, 10)
(360, 10)
(361, 10)
(362, 10)
(362, 10)
(363, 10)
(365, 10)
(365, 10)
(365, 10)
(365, 10)


In [29]:
df = df.drop_duplicates()
print(df.shape)
print(df)
df.to_csv("dataset_testV2.csv", index=False)

(1457, 10)
     language                                             review  voted_up  \
0     english  Welcome to FUN. I enjoyed this game so much, t...      True   
1     english  [h1]The cheapest Far Cry money can buy. [/h1]\...      True   
2     english  The story is nonexistent. You're on an island,...     False   
3     english                                 Poor mans farcry 5      True   
4     english  Welcome to Princeland is a game that is so hig...     False   
...       ...                                                ...       ...   
4292  english  Trying to get anywhere with stats is a pain in...     False   
4293  english  I usually don't write reviews, but I'll make a...      True   
4294  english  It's hard to find an interesting story with go...      True   
4295  english  I had a real blast with this story, its pretty...      True   
4296  english  It's one of the best fantasy game I've ever pl...      True   

      votes_up  votes_funny   weighted_vote_score  c

# ---NEW---

In [30]:
import urllib.request
import pandas as pd
import numpy as np
import json

df = pd.DataFrame(columns=["language", "review", "voted_up", "votes_up", "votes_funny", "weighted_vote_score", "comment_count", "steam_purchase", "received_for_free", "written_during_early_access"])

#game = 240
#game = 4000

games = [837910,2514620,2017950,933756,422100,815740,1295120,1336460,781880,914290,2349580,518908,989560,2062020,300642,1951660,577280,818070,2092500,1515920,1284480,1339380,226453,819980,1319750,2128870,1259230,400140,593420,693623,733670,2180180,511920,1019770,756200,206521,334490,987020,1090090,5688,522870,551160,1906710,2306370,1160243,302340,2319700,1393570,1781760,562040,2710300,585,292230,2024440,853887,1853370,2081500,772710,642805,336040,1681290,369530,1684160,1135591,518848,1828230,2063850,1396940,2015840,1588570,1179260,2623440,313560,2424630,1441714,1621650,346470,1105400,3430,566910,2409940,741590,1099240,1857940,2470710,1521070,2314380,2492080,1240300,2545170,1630080,488700,1689060,2479440,1680973,356744,1917610,396620,1736801,1773050,541890,1571260,415960,269890,11500,439700,2057781,2363960,1114990,1351420,966830,1885090,1989230,1932470,1572960,716940,2534660,2460300,2334760,936850,605660,2068190,2294610,839760,2568860,2673080,845870,2663830,1731930,1195620,1239450,1050560,2001900,1592760,364060,204483,590840,397990,1448010,2589650,518949,387530,2285890,295200,779300,1700470,2340170,211,1159240,268101,714080,407150,409257,1803580,1202034,1290911,2106930,2622870,1498130,729230,940670,1245200,2366470,726510,81107,2269020,2575520,1900860,2455730,2587190,2091850,2404470,1694070,2407764,1766690,1378300,367970,1833960,694060,235580,2165600,1630740,409750,535043,1336482,1930310,2279510,2161920,1105160,67250,1115680,774131,1298220,1925030,2611280,785560,409960,312260,2380190,1183670,1587570,2146500,2554360,1442280,949060,230760,1630601,1205470,1714370,2626870,1026970,2530560,524210,42510,2678060,600630,767,26700,1794760,2318880,917240,1750170,2461430,1005180,349020,623140,2276850,1542910,2580740,1198841,387740,1403740,2518800,2150060,1775000,1852251,379544,864790,2377030,1694170,2314470,1022448,2096830,1227453,527402,32110,1101960,1469620,591740,2015500,1339110,1686390,735540,625730,1404042,1269610,1835810,34344,2231840,358250,353570,2344930,249310,1684780,50414,1086160,269470,1553130,1439340,1285590,1902600,2538200,524042,1672490,2255060,584860,1805059,1492090,331160,2350830,2083130,973270,2209560,1520800,960890,2201560,593900,2190320,2321920,346450,2184560,651140,2127090,1280420,2061290,2169690,1102060,2586420,1934790,1787860,1748472,2163450,1830010,271411,2309890,648110,544230,1827010,1936510,2701470,1485291,2080060,1461854,1135310,2386992,1540490,895370,505015,253831,886140,660940,1791710,491550,766100,2275650,627855,1782480,1063080,389670,851318,1841800,1552561,796860,1427591,457210,1695890,874890,357190,1594170,1619880,550400,721146,2181140,1533250,2082290,1288170,2419627,2483020,597383,1043420,538080,959,583570,1117730,410810,526760,759418,668490,441250,1685970,2383600,988970,2194040,442240,492720,2372580,1343250,311190,527810,1437680,1334790,1658130,1363650,454800,1364410,2666920,15190,2021880,905340,1526820,1674420,1649410,1665400,436070,1652200,24038,2169140,952798,1555442,594220,1004919,1004390,343300,2537490,1392690,80728,1453380,1488890,2622500,573560,1364090,81234,1705249,701420,820500,2253050,2014510,783391,2560510,1795180,81634,952970,48804,2381740,1274850,1150790,2551580,445931,1006890,2431400,633830,951820,1746550,718720,1318820,1736802,1965290,1295440,1377580,227020,81472,2114630,1961400,2067790,965680,1430050,1516930,2141700,2482540,1843500,1671310,1117390,2193600,6550,1466650,527403,222603,2003600,35050,1367770,541930,1175560,650350,1601680,1484380,2011390,2003620,2469360,1495370,1615620,1299660,359480,1449580,751670,726550,362670,1341890,1295920,2191660,889921,2385670,206650,442318,1784780,1257950,662840,747740,1132220,1380170,1774345,2262250,1203482,577080,1476040,2108460,2212875,390371,2259010,2089812,2663080,430890,348470,2501590,2108020,937710,439610,1929650,2069930,3010,1110030]

#with open("games.json") as data:
#    d = json.load(data)
#    data.close()

#for i in range(0, d["response"]["game_count"]):
#    games.append(d["response"]["games"][i]["appid"])

for game in games:

    # ALL WITHIN FOR LOOP OF APPIDS
    with urllib.request.urlopen("http://store.steampowered.com/appreviews/"+str(game)+"?json=1") as url:
        d = json.load(url)
        url.close()


    total_reviews = d["query_summary"]["total_reviews"]
    total_adjust = total_reviews-total_reviews%100

    desired_iter = 100

    if total_adjust > desired_iter*100:
        max_iter = desired_iter
    else:
        max_iter = int(total_adjust/100)

    cursors = []
    cursorURL = ""
    cursor = ""

    print(max_iter)
    
    if max_iter == 0:
        continue
    
    for j in range(0, max_iter):
        if cursor:
            cursorURL = "&cursor="+cursor

        with urllib.request.urlopen("http://store.steampowered.com/appreviews/"+str(game)+"?json=1&num_per_page=100"+cursorURL) as url:
            d = json.load(url)
            url.close()

            if "cursor" in d.keys():
                cursor = urllib.parse.quote(d["cursor"])
            else:
                break

            cursors.append(cursorURL)

    cursors = np.unique(cursors)

    for j in cursors:
        with urllib.request.urlopen("http://store.steampowered.com/appreviews/"+str(game)+"?json=1&num_per_page=100"+j) as url:
            d = json.load(url)
            url.close()

        for k in d["reviews"]:
            df.loc[len(df)] = [k["language"], k["review"], k["voted_up"], k["votes_up"], k["votes_funny"], k["weighted_vote_score"], k["comment_count"], k["steam_purchase"], k["received_for_free"], k["written_during_early_access"]]

        #print(d["query_summary"]["num_reviews"])

print(df.shape)

0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
2
0
10
0
0
28
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
82
0
0
4
0
0
0
0
0
0
0
0
7
0
9
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
3
4
0
0
2
0
0
0
0
0
10
0
0
0
0
0
0
2
0
1
4
0
0
0
0
0
0
0
1
0
0
0
0
0
31
0
5
0
0
0
2
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0

In [31]:
#df = pd.read_csv("dataset.csv")
df = df.drop_duplicates()
print(df.shape)
print(df)
df.to_csv("datasetV5.csv", index=False)

(927, 10)
     language                                             review  voted_up  \
0     english  Welcome to FUN. I enjoyed this game so much, t...      True   
1     english  [h1]The cheapest Far Cry money can buy. [/h1]\...      True   
2     english  The story is nonexistent. You're on an island,...     False   
3     english                                 Poor mans farcry 5      True   
4     english  Welcome to Princeland is a game that is so hig...     False   
...       ...                                                ...       ...   
1441  english  Hero of the Kingdom: The Lost Tales II is the ...      True   
1442  english                                           begendim      True   
1445  english  I asked Bing AI to write a review:\n\nA Dump i...      True   
1446  english  The game itself is very good. The Samurai Warr...     False   
1448  english  I typed rally, cause I wanted to check  DiRT r...      True   

      votes_up  votes_funny   weighted_vote_score  co